In [212]:
import requests
import pandas as pd
import time

import warnings
warnings.filterwarnings('ignore')

In [213]:
api_key = 'Sxwiysbra37brnJK9Fp12Z81vXvU0IEN'

In [214]:
def df_search_item(itemName = '', item_hashtag = ''):
    url = 'https://api.neople.co.kr/df/items'

    params = {
        'limit' : 30,
        'apikey' : api_key,
        'wordType' : 'full'
    }

    if itemName != '':
        params['itemName'] = itemName
    if item_hashtag != '':
        params['hashtag'] = item_hashtag

    return requests.get(url, params=params)

def df_aution_item(itemName, limit=100):
    url = f'https://api.neople.co.kr/df/auction'
    params = {
        'itemName' : itemName,
        'limit' : limit,
        'sort' : 'auctionNo:desc',
        'apikey' : api_key
        
    }
    
    return requests.get(url, params=params)

def df_aution_sold(itemName, limit=100):
    url = f'https://api.neople.co.kr/df/auction-sold'
    params = {
        'itemName' : itemName,
        'limit' : limit,
        'apikey' : api_key
    }
    
    return requests.get(url, params = params)



In [281]:
def check_new_items(check_cols, new_check_lst, isFirst, check_type, itemId = ''):

    
    new_items = []
    
    if check_type == 'regi':

        if isFirst:     # 계속 켜놓지 못하니까 불러와서 확인하기 
            print('Frist Search in REGI')
            auctionIds = get_data_from_db(check_type)

            for check_value in check_cols:
                if str(check_value) not in auctionIds:
                    new_items.append(check_value)
                new_check_lst.append(check_value)   

        
        else:
            for check_value in check_cols:
                if check_value not in new_check_lst:
                    new_items.append(check_value)
                    new_check_lst.append(check_value)
                
    elif check_type == 'saled':
        if isFirst:
            print('Frist Search in SALED')
            db_saled_dates = get_data_from_db(check_type, target = itemId)
            
            for check_value in check_cols:
                if check_value not in db_saled_dates:
                    new_items.append(check_value)
                new_check_lst[itemId].append(check_value)   

        else:
            for check_value in check_cols:
                if check_value not in new_check_lst[itemId] :
                    new_items.append(check_value)
                    new_check_lst[itemId].append(check_value)
                
    return new_items

def regi_item(df, check_type):
    
    if check_type == 'regi':
        use_cols = ['auctionNo', 'regDate', 'itemId', 'count', 'unitPrice']
        
    elif check_type == 'saled':
        use_cols = ['soldDate', 'itemId', 'count', 'unitPrice']
        
    excecuteDB(df[use_cols], check_type)
    

# 경매장에서 아이템 검색
def searching_item(target, check_type):
    
    if check_type == 'regi':
        return pd.DataFrame(df_aution_item(target).json()['rows']) 
    
    elif check_type == 'saled':
        return pd.DataFrame(df_aution_sold(target).json()['rows']) 


def search_and_regi_item(target, new_check_regi_lst, new_check_saled_lst, isFrist, check_type):
    
    df = searching_item(target, check_type)
    
    # 경매장 등록 새로운 아이템 체크
    if check_type == 'regi':
        new_item = df['auctionNo']
        new_item = check_new_items(new_item, new_check_regi_lst, isFrist, check_type)
        df = df[df['auctionNo'].isin(new_item)] # 새롭게 등록된 아이템 필터링

    # 경매장 판매 새로운 아이템 체크
    elif check_type == 'saled':
        new_item = df['soldDate']
        new_item = check_new_items(new_item, new_check_saled_lst, isFrist, check_type, itemId = item_dicts[target])
        df = df[df['soldDate'].isin(new_item)] # 새롭게 등록된 아이템 필터링
        
    if len(df) > 0:
        print(f'{check_type} {target} New {len(new_item)} items')  
    regi_item(df, check_type) # DB등록

In [293]:
import  mysql.connector

def init_mysql():
    aws = mysql.connector.Connect(
        database = 'df',
        host = "dungeonandfighter-db.crnv1am9ifjd.ap-northeast-2.rds.amazonaws.com",
        port = 3306,
        user = 'insert',
        password = '1234'
    )
    return aws

In [285]:
def excecuteDB(df, check_type):
    cur = aws.cursor()
    cols = df.columns
    insert_cols = ",".join(cols)
    
    for idx, row in df.iterrows():
        insert_value = ['"'+str(row[col])+'"' for col in cols]
        insert_value = ",".join(map(str,insert_value))
        
        query = ''
        
        if check_type == 'regi':
            query = 'INSERT INTO ticket_history('+ insert_cols +') VALUES('+ insert_value +');'
            
        elif check_type == 'saled':
            query = 'INSERT INTO sold_history('+ insert_cols +') VALUES('+ insert_value +');'
        
        if query != '':
            cur.execute(query)
        
    aws.commit()
    

In [283]:
# 등록되어있는 기록의 옥션ID정보 가져오기(리스트)
def get_data_from_db(check_type, target = ''):
    cur = aws.cursor()
    
    if check_type == 'regi':
        query = 'SELECT auctionNo FROM ticket_history'
        cur.execute(query)
        return pd.DataFrame(cur.fetchall(), columns={'auctionNo'})['auctionNo'].values
    
    elif check_type == 'saled':
        query = f"Select soldDate from sold_history where itemId like '{target}'"
        cur.execute(query)
        return pd.DataFrame(cur.fetchall(), columns={'soldDate'})['soldDate'].values

In [228]:
search_item = '특별한 황금 주화'

danjin_gold_token = pd.DataFrame(df_search_item(itemName=search_item).json()['rows'])
danjin_gold_token = danjin_gold_token[danjin_gold_token['itemName'].str.contains('단진')]
danjin_gold_token = danjin_gold_token[danjin_gold_token['itemName'].str.contains('황금 주화')]

search_names = danjin_gold_token['itemName'].values
search_ids = danjin_gold_token['itemId'].values

item_dicts = {item_name : item_id for item_name, item_id in zip(search_names, search_ids)}

item_dicts

{'대형 단진의 특별한 황금 주화 상자': 'c4df9933c11b3399b21f362ba4395523',
 '소형 단진의 특별한 황금 주화 상자': '83f74cef66c5fa5c1d30137ed125941c'}

In [ ]:
new_check_regi_lst = []
new_check_saled_lst = {search_id : [] for search_id in search_ids}
search_type = ['regi', 'saled']
isFrist = True

aws = init_mysql()

while True:
    for target in item_dicts.keys():
        try:
            for check_type in search_type:
                search_and_regi_item(target, new_check_regi_lst, new_check_saled_lst, isFrist, check_type = check_type)
        except Exception as e:
            print(e)
            time.sleep(2)
            
    isFrist = False
    time.sleep(0.5)

Frist Search in REGI
Frist Search in SALED
saled 대형 단진의 특별한 황금 주화 상자 New 8 items
['2023-04-03 11:57:06', '2023-04-03 11:57:04', '2023-04-03 11:53:22', '2023-04-03 11:50:39', '2023-04-03 11:47:22', '2023-04-03 11:47:13', '2023-04-03 11:47:11', '2023-04-03 11:45:37']
Frist Search in REGI
regi 소형 단진의 특별한 황금 주화 상자 New 2 items
[1234020396, 1234020163]
Frist Search in SALED
saled 소형 단진의 특별한 황금 주화 상자 New 1 items
['2023-04-03 11:41:39']
regi 대형 단진의 특별한 황금 주화 상자 New 1 items
[1234021896]
saled 대형 단진의 특별한 황금 주화 상자 New 1 items
['2023-04-03 12:03:34']
saled 대형 단진의 특별한 황금 주화 상자 New 1 items
['2023-04-03 12:11:00']
regi 소형 단진의 특별한 황금 주화 상자 New 1 items
[1234023153]
regi 소형 단진의 특별한 황금 주화 상자 New 1 items
[1234023639]
regi 소형 단진의 특별한 황금 주화 상자 New 2 items
[1234023674, 1234023655]
regi 대형 단진의 특별한 황금 주화 상자 New 2 items
[1234023688, 1234023629]
regi 대형 단진의 특별한 황금 주화 상자 New 1 items
[1234023938]
regi 소형 단진의 특별한 황금 주화 상자 New 1 items
[1234023982]
regi 소형 단진의 특별한 황금 주화 상자 New 2 items
[1234024138, 1234024086]
regi 대형

saled 소형 단진의 특별한 황금 주화 상자 New 1 items
['2023-04-03 15:55:46']
regi 대형 단진의 특별한 황금 주화 상자 New 1 items
[1234053648]
regi 소형 단진의 특별한 황금 주화 상자 New 1 items
[1234053749]
saled 소형 단진의 특별한 황금 주화 상자 New 1 items
['2023-04-03 15:57:04']
regi 대형 단진의 특별한 황금 주화 상자 New 1 items
[1234054249]
regi 대형 단진의 특별한 황금 주화 상자 New 1 items
[1234054690]
regi 소형 단진의 특별한 황금 주화 상자 New 1 items
[1234054669]
regi 대형 단진의 특별한 황금 주화 상자 New 1 items
[1234055014]
saled 소형 단진의 특별한 황금 주화 상자 New 1 items
['2023-04-03 16:06:25']
regi 대형 단진의 특별한 황금 주화 상자 New 1 items
[1234056453]
saled 소형 단진의 특별한 황금 주화 상자 New 1 items
['2023-04-03 16:13:44']
regi 소형 단진의 특별한 황금 주화 상자 New 1 items
[1234056507]
saled 소형 단진의 특별한 황금 주화 상자 New 1 items
['2023-04-03 16:18:39']
regi 대형 단진의 특별한 황금 주화 상자 New 1 items
[1234057786]
saled 소형 단진의 특별한 황금 주화 상자 New 1 items
['2023-04-03 16:24:58']
saled 소형 단진의 특별한 황금 주화 상자 New 1 items
['2023-04-03 16:25:38']
regi 소형 단진의 특별한 황금 주화 상자 New 1 items
[1234058700]
regi 대형 단진의 특별한 황금 주화 상자 New 1 items
[1234059469]
regi 대형 단진의 특별한 

In [289]:
aws.close()